From this program the user can input the following derived KPIs and  behavioural parameters of lines and generate events events that results those derived KPIs.



**Inputs**
*   Operators
*   Helpers
*   Ironman
*   QC
*   Schedule showing work hour length
*   SMV
*   Efficiency
*   Defective rate


**Behavioural hyperparametrs you can adjust**
*   ftt_ratio : Ratio of FTTs from FTT+ Rectified ina  line
*   direct_rejects : Number of pcs being directly rejeceted without attempted rectification
*   defect_increment : Difference between the number of defective and defects
*   target_increment : Offset to add to FTT + Rectified + Reject to generate
planned_qty


**KPIs being fixed as a result**

*   Manpower
*   Man Minutes
*   SAH
*   Actual output
*   Trend
*   Checked_pcs
*   FTT
*   Reworked
*   Defective
*   Reject
*   Checked pcs
*   FTT %
*   Reject rate%
*   DHU
*   Planned Qty
*   RTT





In [169]:
# all imports
import pandas as pd
import numpy as np
import random
from random import randint
from google.colab import drive

**Note that the plan_qty and produced_mins columns in the plan are going to be altered according to the derived KPIs you provide in a later stage.**

In [170]:
# Read plan and schedule data from drive
drive.mount('/content/drive')
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [171]:
plan

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1100,660,23,5,0,0,NaN,8.28,NaN
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,800,660,20,5,0,0,NaN,8.28,NaN
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1000,660,23,5,0,0,NaN,10.11,NaN
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,900,660,23,5,0,0,NaN,8.21,NaN
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,1200,660,26,8,0,0,NaN,8.28,NaN
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,500,660,12,5,0,0,NaN,8.28,NaN
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,600,660,12,5,0,0,NaN,9.45,NaN
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,500,660,10,5,0,0,NaN,8.28,NaN


In [172]:
# Generate the skeleton of the target table
repeated_plan_date =  plan['date'].repeat(len(schedule)).reset_index(drop=True)
repeated_module_id = plan['module_id'].repeat(len(schedule)).reset_index(drop=True)
hours = list(range(len(schedule)+1))
hours.remove(0)
repeated_hour = hours*len(plan)
targets = pd.DataFrame({'Date': repeated_plan_date, 'Line' : repeated_module_id,
                        'Hour': repeated_hour})


Let us generate reasonable manpower-realated columns by refering the plan data.
**Remember to change the Manpower definition according to the factory.**

In [173]:
def generate_man_power(targets, plan, manpower_regulator):
    """
    Generate counts for each hour of each day for each line group of a DataFrame
    to match the target counts by distributing the target sum among all rows in
    the hour + date + line group.

    :param target: DataFrame containing targets.
    :param plan: DataFrame with planned values per day.
    :param manpower_regulator : The higest possible member variation
    :return: DataFrame with generated manpower counts.
    """
    # Merge the dataframes on the 'hour, date, line' columns
    merged_df = targets.merge(plan[['date', 'module_id',  'operator', 'helper',
       'ironman', 'qc', 'manpower', 'smv']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
    merged_df.drop(columns=['date', 'module_id'], inplace=True)


    merged_df['Operator'] = np.maximum((merged_df['operator'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    merged_df['Helper'] = np.maximum((merged_df['helper'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    merged_df['Iron man'] = np.maximum((merged_df['ironman'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    merged_df['QC'] = np.maximum((merged_df['qc'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    # Define this as factory defines
    merged_df['Man power'] = merged_df['Operator'] + merged_df['Helper']
    merged_df.drop(columns=['operator', 'helper', 'ironman', 'qc','manpower'], inplace=True)
    return merged_df



In [174]:
merged_df = generate_man_power(targets,plan, manpower_regulator=1)
merged_df.head(100)

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28
...,...,...,...,...,...,...,...,...,...
95,2024-07-27,F1M8,8,9.45,12,5,0,0,17
96,2024-07-27,F1M8,9,9.45,12,5,0,0,17
97,2024-07-27,F1M8,10,9.45,12,5,0,0,17
98,2024-07-27,F1M8,11,9.45,12,5,0,0,17


In [175]:
#Convert start_time and end_time to datetime
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:%S')
schedule['end_time'] = pd.to_datetime(schedule['end_time'], format='%H:%M:%S')

# Calculate the duration between start_time and end_time in hours
schedule['clock_mins'] = (schedule['end_time'] - schedule['start_time']).dt.total_seconds() / 60
schedule

# Sum the clock_hours to get total hours in clock time
# total_clock_hours = schedule['clock_hours'].sum()
# total_clock_hours

,hour,start_time,end_time,clock_mins
0,1,1900-01-01 07:30:00,1900-01-01 08:30:00,60.0
1,2,1900-01-01 08:30:00,1900-01-01 09:30:00,60.0
2,3,1900-01-01 09:30:00,1900-01-01 10:30:00,60.0
3,4,1900-01-01 10:30:00,1900-01-01 11:30:00,60.0
4,5,1900-01-01 11:30:00,1900-01-01 13:00:00,90.0
5,6,1900-01-01 13:00:00,1900-01-01 14:00:00,60.0
6,7,1900-01-01 14:00:00,1900-01-01 15:00:00,60.0
7,8,1900-01-01 15:00:00,1900-01-01 16:00:00,60.0
8,9,1900-01-01 16:00:00,1900-01-01 17:00:00,60.0
9,10,1900-01-01 17:00:00,1900-01-01 18:00:00,60.0


In [176]:
# Now we merge the schedule with the merged_df to find the length of each working hour
derived_kpis = merged_df.merge(schedule[['hour', 'clock_mins']],
                              left_on=['Hour'],
                              right_on= ['hour'], how='left')
derived_kpis.drop(columns=['hour'], inplace=True)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60.0
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60.0
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60.0
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60.0
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90.0
...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60.0
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60.0
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60.0
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60.0


In [177]:
np.random.seed(42)

Now let us add an efficiency column with values between 70-100 and a defective_rate column between 3-12.

**If needed the user can go and change this in the sheet.**
ftt_ratio indicates the propotion of ftts from ftt+ rectified qty.
( that is, ftt/ all actuals)



In [178]:
# Autogenerate efficiency, defective rate and ftt_ratio

derived_kpis['target_efficiency'] = np.random.randint(70, 100, size=len(derived_kpis))
derived_kpis['target_defective_rate'] = np.random.randint(3, 20, size=len(derived_kpis))
derived_kpis['ftt_ratio'] = np.random.uniform(0.8, 0.98, size=len(derived_kpis))
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60.0,76,11,0.887614
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60.0,89,10,0.963098
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60.0,98,14,0.878191
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60.0,84,4,0.863014
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90.0,80,3,0.916119
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60.0,76,14,0.922690
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60.0,95,16,0.861312
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60.0,78,8,0.846925
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60.0,97,8,0.889287


In [179]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)

In [180]:
# Reaload everything from drive incase the user edited any derived KPI values.
drive.mount('/content/drive', force_remount=True)
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule", "derived_kpis"])

# Access individual sheets using sheet names
#plan = xls["plan"]
#schedule = xls["schedule"]
derived_kpis = xls["derived_kpis"]

Mounted at /content/drive


In [181]:
plan

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1100,660,23,5,0,0,NaN,8.28,NaN
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,800,660,20,5,0,0,NaN,8.28,NaN
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1000,660,23,5,0,0,NaN,10.11,NaN
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,900,660,23,5,0,0,NaN,8.21,NaN
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,1200,660,26,8,0,0,NaN,8.28,NaN
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,500,660,12,5,0,0,NaN,8.28,NaN
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,600,660,12,5,0,0,NaN,9.45,NaN
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,500,660,10,5,0,0,NaN,8.28,NaN


In [182]:
# Derive the Actual count using efficiency
derived_kpis['ftt_rectified_sum'] = (derived_kpis['target_efficiency'] *
                                    derived_kpis['clock_mins'] *
                                    derived_kpis['Man power'])/ (derived_kpis['smv'] * 100)
derived_kpis['ftt_rectified_sum'] = derived_kpis['ftt_rectified_sum'].round()
derived_kpis['ftt'] = derived_kpis['ftt_rectified_sum'] * derived_kpis['ftt_ratio']
derived_kpis['ftt'] = derived_kpis['ftt'].round()
derived_kpis['rectified'] = derived_kpis['ftt_rectified_sum'] - derived_kpis['ftt']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio,ftt_rectified_sum,ftt,rectified
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60,76,11,0.887614,149.0,132.0,17.0
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,89,10,0.963098,181.0,174.0,7.0
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,98,14,0.878191,199.0,175.0,24.0
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,84,4,0.863014,170.0,147.0,23.0
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90,80,3,0.916119,243.0,223.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,76,14,0.922690,110.0,101.0,9.0
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,95,16,0.861312,138.0,119.0,19.0
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,78,8,0.846925,113.0,96.0,17.0
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,97,8,0.889287,141.0,125.0,16.0


Defective rate = (Defective  x 100 )/ (FTT + Defective + Reject)

We know that Reject pcs should be greater than or equal Defective - Rectified.

**Reject (min) =  Defective - Rectified**


We also know that the Rectified should be less than or equal to Defective.

**Defective(min) = Rectified **


Defective rate = (Defective  x 100 )/ (FTT + Defective + Defective - Rectified + Delta)

Defective goes down => numerators goes down 100 times and denominator goes down 2* => defective rate goes down. Minimum defective rate is given at minimum Defectives i.e. Defective = Rectified


Defective rate (min) = (Rectified x 100) / (FTT + 2 x Rectified - Rectified +   Delta(max))


Defective rate (min) = (Rectified x 100) / (FTT + Rectified +  Delta(max))


In [183]:
# This indicates that each line may directly reject upto 6 garments that are not
# added as defectives first
max_direct_rejects = 6
derived_kpis['direct_rejects']= np.random.randint(0, max_direct_rejects, size=len(derived_kpis))

# Implementing minimum defective rate where defectives = rectified
derived_kpis['defective_rate_min'] = np.ceil((derived_kpis['rectified']*100 )/ (derived_kpis['ftt'] + derived_kpis['rectified'] + derived_kpis['direct_rejects'])).astype(int)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio,ftt_rectified_sum,ftt,rectified,direct_rejects,defective_rate_min
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60,76,11,0.887614,149.0,132.0,17.0,1,12
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,89,10,0.963098,181.0,174.0,7.0,4,4
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,98,14,0.878191,199.0,175.0,24.0,2,12
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,84,4,0.863014,170.0,147.0,23.0,4,14
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90,80,3,0.916119,243.0,223.0,20.0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,76,14,0.922690,110.0,101.0,9.0,3,8
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,95,16,0.861312,138.0,119.0,19.0,2,14
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,78,8,0.846925,113.0,96.0,17.0,2,15
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,97,8,0.889287,141.0,125.0,16.0,3,12


**At this point, the programe will provide a symentically correct target_defective rate (min) value and ask the user to alter. You can go to the excel sheet and edit values. Afterwards the sheet will be reloaded and any remaining misfits will be set to the default minimum.**

In [184]:
# Find rows where targets are semantically unccceptable
for index, row in derived_kpis.iterrows():
    if row['target_defective_rate'] < row['defective_rate_min']:
        print(f"please change target_defective_rate value at row  {index} to a value larger than or equal to {row['defective_rate_min']}")

please change target_defective_rate value at row  0 to a value larger than or equal to 12
please change target_defective_rate value at row  3 to a value larger than or equal to 14
please change target_defective_rate value at row  4 to a value larger than or equal to 9
please change target_defective_rate value at row  7 to a value larger than or equal to 16
please change target_defective_rate value at row  10 to a value larger than or equal to 7
please change target_defective_rate value at row  11 to a value larger than or equal to 20
please change target_defective_rate value at row  13 to a value larger than or equal to 12
please change target_defective_rate value at row  16 to a value larger than or equal to 7
please change target_defective_rate value at row  18 to a value larger than or equal to 14
please change target_defective_rate value at row  20 to a value larger than or equal to 10
please change target_defective_rate value at row  22 to a value larger than or equal to 19
please

In [185]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)



In [186]:
# Reload the excel sheet where user ay have edited target_defective-rate
drive.mount('/content/drive', force_remount=True)
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule", "derived_kpis"])

# Access individual sheets using sheet names
# plan = xls["plan"]
# schedule = xls["schedule"]
derived_kpis = xls["derived_kpis"]


Mounted at /content/drive


In [187]:
plan

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1100,660,23,5,0,0,NaN,8.28,NaN
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,800,660,20,5,0,0,NaN,8.28,NaN
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1000,660,23,5,0,0,NaN,10.11,NaN
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,900,660,23,5,0,0,NaN,8.21,NaN
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,1200,660,26,8,0,0,NaN,8.28,NaN
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,500,660,12,5,0,0,NaN,8.28,NaN
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,600,660,12,5,0,0,NaN,9.45,NaN
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,500,660,10,5,0,0,NaN,8.28,NaN


In [188]:
# Set any target_defective_rate values that are still incorrect to the default minimum value.
for index, row in derived_kpis.iterrows():
    if row['target_defective_rate'] < row['defective_rate_min']:
        derived_kpis.at[index, 'target_defective_rate'] = row['defective_rate_min']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio,ftt_rectified_sum,ftt,rectified,direct_rejects,defective_rate_min
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60,76,12,0.887614,149,132,17,1,12
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,89,10,0.963098,181,174,7,4,4
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,98,14,0.878191,199,175,24,2,12
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,84,14,0.863014,170,147,23,4,14
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90,80,9,0.916119,243,223,20,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,76,14,0.922690,110,101,9,3,8
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,95,16,0.861312,138,119,19,2,14
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,78,15,0.846925,113,96,17,2,15
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,97,12,0.889287,141,125,16,3,12


In [189]:
# Now derive defective, reject and defect values
derived_kpis['defective'] = (derived_kpis['target_defective_rate'] * (derived_kpis['ftt'] - derived_kpis['rectified'] + derived_kpis['direct_rejects']))/(100 - 2*derived_kpis['target_defective_rate'])
derived_kpis['defective'] = derived_kpis['defective'].round()
derived_kpis['reject'] = derived_kpis['defective'] - derived_kpis['rectified'] + derived_kpis['direct_rejects']
# derived_kpis['test'] = derived_kpis['defective'] * 100 / (derived_kpis['ftt'] + derived_kpis['defective'] + derived_kpis['reject'])
# derived_kpis['test'] = derived_kpis['test'].round()
derived_kpis['defect_increment'] = np.random.randint(0, 4, size=len(derived_kpis))
derived_kpis['defect'] = derived_kpis['defective'] + derived_kpis['defect_increment']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,ftt_ratio,ftt_rectified_sum,ftt,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60,...,0.887614,149,132,17,1,12,18.0,2.0,1,19.0
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,...,0.963098,181,174,7,4,4,21.0,18.0,2,23.0
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,...,0.878191,199,175,24,2,12,30.0,8.0,0,30.0
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,...,0.863014,170,147,23,4,14,25.0,6.0,0,25.0
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90,...,0.916119,243,223,20,3,9,23.0,6.0,1,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,...,0.922690,110,101,9,3,8,18.0,12.0,1,19.0
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,...,0.861312,138,119,19,2,14,24.0,7.0,1,25.0
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,...,0.846925,113,96,17,2,15,17.0,2.0,2,19.0
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,...,0.889287,141,125,16,3,12,18.0,5.0,0,18.0


In [190]:
plan

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1100,660,23,5,0,0,NaN,8.28,NaN
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,800,660,20,5,0,0,NaN,8.28,NaN
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1000,660,23,5,0,0,NaN,10.11,NaN
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,900,660,23,5,0,0,NaN,8.21,NaN
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,NaN,9.45,NaN
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,1200,660,26,8,0,0,NaN,8.28,NaN
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,500,660,12,5,0,0,NaN,8.28,NaN
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,600,660,12,5,0,0,NaN,9.45,NaN
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,500,660,10,5,0,0,NaN,8.28,NaN


In [191]:
# Edit plan qty and produced_mins
summed_kpis = derived_kpis.groupby(['Date', 'Line']).agg({'ftt': 'sum','rectified': 'sum','reject': 'sum'
}).reset_index()

# Merge the summed data with the plan DataFrame on 'date' and 'module_id'
plan = pd.merge(plan, summed_kpis, left_on=['date', 'module_id'], right_on=['Date', 'Line'] ,how='left')

target_increment = 20
plan['qty'] = plan['ftt'] + plan['rectified'] + plan['reject'] + np.random.randint(0, 20, size=len(plan))
plan['qty'] = plan['qty'].astype(int)
plan['produced_mins'] = plan['qty'] * plan['smv']

plan.drop(columns=['Date', 'Line', 'ftt', 'rectified', 'reject'], inplace=True)
plan['manpower'] = plan['operator'] + plan['helper'] + plan['ironman'] + plan['qc']



In [192]:
plan


,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,2114,660,23,5,0,0,28,8.28,17503.92
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1861,660,23,5,0,0,28,9.45,17586.45
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,1815,660,20,5,0,0,25,8.28,15028.20
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1809,660,23,5,0,0,28,10.11,18288.99
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,2191,660,23,5,0,0,28,8.21,17988.11
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,1837,660,23,5,0,0,28,9.45,17359.65
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,2687,660,26,8,0,0,34,8.28,22248.36
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1225,660,12,5,0,0,17,8.28,10143.00
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1132,660,12,5,0,0,17,9.45,10697.40
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,1130,660,10,5,0,0,15,8.28,9356.40


In [193]:
# Add TPPM
derived_kpis = derived_kpis.merge(plan[['date', 'module_id', 'qty', 'mins']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
derived_kpis['TPPM'] = derived_kpis['qty'] / derived_kpis['mins']
derived_kpis.drop(columns=['qty', 'mins'], inplace=True)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect,date,module_id,TPPM
0,2024-07-26,F1M7,1,8.28,23,4,0,0,27,60,...,17,1,12,18.0,2.0,1,19.0,2024-07-26,F1M7,3.20303
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,...,7,4,4,21.0,18.0,2,23.0,2024-07-26,F1M7,3.20303
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,...,24,2,12,30.0,8.0,0,30.0,2024-07-26,F1M7,3.20303
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,...,23,4,14,25.0,6.0,0,25.0,2024-07-26,F1M7,3.20303
4,2024-07-26,F1M7,5,8.28,23,5,0,0,28,90,...,20,3,9,23.0,6.0,1,24.0,2024-07-26,F1M7,3.20303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,...,9,3,8,18.0,12.0,1,19.0,2024-08-01,F1M8,2.15303
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,...,19,2,14,24.0,7.0,1,25.0,2024-08-01,F1M8,2.15303
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,...,17,2,15,17.0,2.0,2,19.0,2024-08-01,F1M8,2.15303
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,...,16,3,12,18.0,5.0,0,18.0,2024-08-01,F1M8,2.15303


In [194]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)

In [195]:
# Rename to suit the Traget KPI structure

derived_kpis.rename(columns={'smv': 'SMV', 'ftt': 'FTT', 'defective' : 'Defective', 'defect': 'Defect', 'rectified': 'Rectified', 'reject': 'Reject'}, inplace=True)

derived_kpis = derived_kpis[['Date','Line','Hour','FTT','SMV','TPPM','Defective','Defect','Rectified','Reject','Operator','Helper','Man power']]


derived_kpis['Defect'] = derived_kpis['Defect'].astype(int)
derived_kpis['Defective'] = derived_kpis['Defective'].astype(int)
derived_kpis['Reject'] = derived_kpis['Reject'].astype(int)

<ipython-input-195-8eb1392a8333>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derived_kpis['Defect'] = derived_kpis['Defect'].astype(int)
<ipython-input-195-8eb1392a8333>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derived_kpis['Defective'] = derived_kpis['Defective'].astype(int)
<ipython-input-195-8eb1392a8333>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [196]:
derived_kpis.head(50)

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-07-26,F1M7,1,132,8.28,3.203030,18,19,17,2,23,4,27
1,2024-07-26,F1M7,2,174,8.28,3.203030,21,23,7,18,23,5,28
2,2024-07-26,F1M7,3,175,8.28,3.203030,30,30,24,8,23,5,28
3,2024-07-26,F1M7,4,147,8.28,3.203030,25,25,23,6,23,5,28
4,2024-07-26,F1M7,5,223,8.28,3.203030,23,24,20,6,23,5,28
5,2024-07-26,F1M7,6,144,8.28,3.203030,37,39,12,25,23,5,28
6,2024-07-26,F1M7,7,190,8.28,3.203030,15,17,9,10,23,5,28
7,2024-07-26,F1M7,8,154,8.28,3.203030,30,30,29,5,23,5,28
8,2024-07-26,F1M7,9,137,8.28,3.203030,23,23,17,6,23,5,28
9,2024-07-26,F1M7,10,174,8.28,3.203030,20,21,19,4,23,5,28


In [197]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='target_kpis', index=False)

In [198]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    plan.to_excel(writer, sheet_name='plan', index=False)